In [21]:
import re,glob,datetime

import pandas as pd
import networkx as nx
import itertools as it

def dateheure(f="%y/%m/%d %H:%M"):
    return datetime.datetime.utcnow().strftime(f) # %y%m%d%H%M


In [23]:
repSamples=u"/Users/gilles/pCloud Drive/FOD/GB/2015-Data/Samples/Verbes3/"
fileSample=u"MGC-10Ko-02-T10000-F3700-Tirage.pkl"
dfSample=pd.read_pickle(repSamples+fileSample)
repJouets=u"/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/Jouets/"
fileJouet=repJouets+"Jouet-%s.csv"%dateheure("%y%m%d-%H%M")
fileJouet

u'/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/Jouets/Jouet-200911-0205.csv'

# Extraction d'un sous-échantillon
- choisir les *nbFormes* les plus fréquentes (indépendamment du tirage)
- former les paradigmes
- regrouper les colonnes en morphomes
- choisir les *nbMorphomes* les plus fréquents
- extraire les paradigmes des lexèmes qui ont au moins une des cases morphomiques remplie
- extraire les paradigmes des lexèmes qui ont au moins une paire de cases morphomiques remplies

=> *nbMorphomes* choisis  
=> sous-échantillon, sous-échantillon avec paires

In [71]:
nbFormes=85
nbSelCols=3
df=dfSample.sort_values("freq",ascending=False).head(nbFormes)

In [72]:
dfPar=pd.pivot_table(df,values="phono",columns="case",aggfunc=",".join,index="lexeme")

In [73]:
parCols=dfPar.columns.tolist()
# dfPar["ii1P"]

In [74]:
def compareCols(c1,c2,dfPar):
    df=dfPar.copy()
    nbEq=sum(df[c1]==df[c2])
    df["c1"]=df[c1]
    df["c2"]=df[c2]
    df["c1c2"]=df.pop("c1").fillna(df.pop("c2"))
    df["c1"]=df[c1]
    df["c2"]=df[c2]
    df["c2c1"]=df.pop("c2").fillna(df.pop("c1"))
    return all(df["c1c2"].dropna()==df["c2c1"].dropna()),nbEq

In [75]:
graphe=nx.Graph()
for c1,c2 in it.combinations(parCols,2):
    colEq,nbEq=compareCols(c1,c2,dfPar)
    if colEq:
        graphe.add_edge(c1,c2,weight=nbEq)

In [76]:
def scoreClique(clique):
    nbComb=0
    score=0
    for c1,c2 in it.combinations(clique,2):
        nbComb+=1
        score+=graphe[c1][c2]["weight"]
    return score/float(nbComb)

In [77]:
cliques=list(nx.algorithms.clique.find_cliques(graphe))
scoresCliques=[]
for clique in cliques:
    scoresCliques.append((scoreClique(clique),clique))
scoresCliques=sorted(scoresCliques,key=lambda x: x[0],reverse=True)
scoresCliques

[(8.0, [u'pi2S', u'pi3S']),
 (2.0, [u'ii3S', u'ii1S']),
 (0.0, [u'pi1S', u'fi3S']),
 (0.0, [u'pi1S', u'ps3S']),
 (0.0, [u'pi1P', u'ppMS']),
 (0.0, [u'pi1P', u'ai3S', u'ps3S']),
 (0.0, [u'pi1P', u'ai3S', u'ii1S']),
 (0.0, [u'pi1P', u'ai3S', u'fi3S']),
 (0.0, [u'pi1P', u'ai3S', u'pi3P']),
 (0.0, [u'pi1P', u'ai3S', u'pc3S'])]

In [78]:
morphomes=[]
for score,clique in scoresCliques:
    if score>=1:
        cliqueInterMorphomes=set(clique)&set([c for m in morphomes for c in m])
        if cliqueInterMorphomes==set():
            morphomes.append((clique))
        else:
            if len(clique)-len(cliqueInterMorphomes)>2:
                print "maybe try a sub-clique",clique,cliqueInterMorphomes
morphomes

[[u'pi2S', u'pi3S'], [u'ii3S', u'ii1S']]

In [79]:
msp2omp={}
omp2msp={}
morCols=parCols[:]
for morphome in morphomes:
    for case in morphome:
        msp2omp[case]=morphome[0]
        if morphome[0] not in omp2msp:
            omp2msp[morphome[0]]=[]
        omp2msp[morphome[0]].append(case)
        morCols.remove(case)
for case in morCols:
    msp2omp[case]=case
    omp2msp[case]=[case]

In [80]:
def formes2Morphomes(df):
    lDF=df.copy()
    for c in msp2omp:
        if c!=msp2omp[c]:
            lDF[msp2omp[c]]=lDF.pop(msp2omp[c]).fillna(lDF.pop(c))
    return lDF

In [81]:
dfMorPar=formes2Morphomes(dfPar)
subCases=dfMorPar.count().sort_values(ascending=False).head(nbSelCols).index.tolist()
dfMorPar[subCases].dropna(thresh=2)

case,pi2S,inf,ppMS
lexeme,,,
aller,va,ale,NaN
avoir,a,avwar,y
devoir,dwa,NaN,dy
dire,di,dir,di
faire,fE,fEr,fE
passer,pas,pase,pase
pouvoir,p2,NaN,py
prendre,NaN,prâdr,pri
savoir,sE,savwar,NaN


In [232]:
dfMorPar[subCases].to_csv(path_or_buf=fileJouet,sep=";",encoding="utf8")

In [70]:
dfMorPar[dfMorPar.index.str.contains("er$")][subCases].dropna(thresh=1)

case,pi2S,inf,pi1S
lexeme,,,
aimer,Em,NaN,NaN
aller,va,ale,vE
arriver,ariv,NaN,NaN
chercher,NaN,SErSe,NaN
parler,NaN,parle,NaN
passer,pas,pase,NaN
penser,pâs,NaN,NaN
regarder,r6gard,NaN,NaN
trouver,NaN,truve,NaN
